
<h1 style="border: 2px solid black; padding: 15px; border-radius: 12px;" align='center'>Cours IA et Applications</h1>    

<h2 align='center'> Analyse de textes en Python : premier pas </h2>

<h3 align='center'> Jordy Palafox </h3>
<h3 align='center'> Ing2 GSI/MI - 2023/2024 </h3>
      
      
<div style="display:flex"> 
    <img src="cytech.png", style="width:250px;height:50"> 
    <img src="cy.jpg", style="width:300px;height:100px"> 
</div> 

Inspiré largement du livre Python pour le Data Scientist, Emmanuel Jakobowicz, Edition Dunod.


Quelques outils classiques sur l'analyse textuelle sont :
+ *String* , interne à Python pour des actions particulières sur la class str
+ *NLTK* (Natural Language Toolkit) pour le prétraitement de textes avec fonctions et bases de données particulières
+ *Scikit-Learn* pour une approche statistique des transformatins
+ *Spacy* https://spacy.io/ qui vient compléter NLTK, plus récent et très complet, avec  des modèles etc...

In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# Prétraitrement des données

Ou comment transformer des données brutes en des données exploitables, structurées.

## Etape 1 : la tokenisation

On va récupérer quelques données (Scraping) de Wikipédia à l'aide de la libraire Beautiful-Soup https://www.crummy.com/software/BeautifulSoup/

In [ ]:
from bs4 import BeautifulSoup
import urllib.request

# On récupère maintenant les données d'une page
reponse = urllib.request.urlopen('https://fr.wikipedia.org/wiki/Python_(langage)')

# extraction du texte en html
html = reponse.read()

# On utilise un objet de la classe BeautifulSoup pour traiter du code html
soup = BeautifulSoup(hmlt, 'html5lib')

# on récupère tout le code à partir de la balise dont la suite nous intéresse
tag = soup.find('div', {'class' : 'mw-parset-output'})

# on extrait le texte du code
text = tag.text
print(type(text), len(text), sep=',')

In [ ]:
f"On a donc récupéré une chaine de caractères composés de {len(text)} catactères".

# Il faut maintenant extraire les mots de cette chaîne

tokens = ntlk.word_tokenize(text.lower(), language='french')
print(type(tokens), len(tokens))

f"On a une liste de {len(tokens)} tokens donc de groupe de caractères en minuscule"

# On regarde souvent les fréquences de ces tokens

freq = ntlk.FreqDist(tokens)

# Les 10 mots les plus communs 
freq.most_common(10)

## Etape 2 : Nettoyer le texte

On voit que le texte donc de nombreux mots qui ne servent pas à le distinguer (les articles par exemple)
Ces mots sont appelés **stopwords** et généralement on les supprime.
Les libraires comme NLTK ou SPacy proposent des dictionnaires de stopwords pour différentes langagues.

Il y a aussi le problème de la ponctuation.


In [ ]:
from nltk.corpus import stopwords
import string

# Créons une liste de mots + mots que l'on ajoute nous même + ponctuation

sr = stopwords.words('french') + ['les', 'a', 'il', '+', ',','<', '>', "''"]+list(string.punctuation)

# On fait le ménage dans les tokens 

tokens_propres = [i for in tokens if i not in sr]

# Puis on regarde à nouveau les mots les plus fréquents
freq = ntlk.FreqDist(tokens_propres)
freq.plot(20, tile = 'Fréquence des mots dans la page')

D'autres outils utiles : sur ntlk, *sent_tokenize* extrait les phrases, WordNet permet de récupérer les synonymes en francais et réduire ainsi le nombre de variables.


Nous allons pouvoir mettre maintenant en place un modèle prédictif puisqu'on sait traiter des données !

# Premier modèle prédictif pour du NLP

On va utiliser un exemple concret, des données (téléchargeables ici https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset ) pour construire un filtre anti spam de sms.

In [ ]:
import pandas as pd

# Lecture des données
sms = pd.read_table( ' ..... ')

# Il est intéressant de regarder les labels donc le nombre de sms non spams et de spams 
# il y a souvent la problématique de rencontrer des classes déséquilibrées qu'il faut prendre en compte

sms['class'].value_counts()

Les données sont textuelles donc non structurées. Pour leur donner une structure nous allons utiliser les deux mêmes suivantes : 
+ *CountVectorizer* qui transforme un document en une matrice de comptage pour chaque mot du texte. Une ligne correspond à un document et une colonne à un mot. Attention, cela créée des matrices sparses (creuses) donc remplies de zéro, Numpy permet de stocker sous un format optimisé ce type de matrice (optimisé en taille de stockage et temps d'accès)

+ *TF/IDF* partant du comptage, on va passer aux fréquence, un mot fréquent est moins discriminant ou caractéristique qu'un mot rare.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# on encode la variable à prédire en une variable binaire 
encode_y = LabelEncoder()
y = encode_y.fit_transform(sms['class'])

# On sépare en données d'entraînement et de test

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(sms['sms'], y, test_size=0.2)

# On applique tf_idf ensuite !
from sklearn.feature_selection.text import TfidfVectorizer
trans_vect = TfidfVectorizer()

x_train_trans = trans_vect.fit_transform(x_train)
x_test_trans = trans_vect.transform(x_test)

On va ensuite applqiuer des classifieurs classiques (pas des réseaux de neurones mais efficace quand même en général !)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# on instantie les modèles
bayes = MultinomialNB()
svm = SVC()

bayes.fit(x_train_trans, y_train)
svm.fit(x_train_trans, y_train)

In [ ]:
# Il faut ensuite vérifier la qualité des modèles

from sklearn.metrics import roc_auc_score, accuracy_score
print(f'Accuracy pour le modèle NaiveBayes : {accuracy_score(y_test, bayes.predict(x_test_trans))}')


Comparer au modèle SVM.

Dans une logique de mise en production, donc d'automatisation, on va construire une pipeline de traitement pour de futurs messages arrivant : 

In [ ]:
from sklearn.pipeline import make_pipeline
import numpy as np

# créons une pipeline de traitement du text
pipe_text = make_pipeline(TfidfVectorizer(), MultinomialNB())

# On réentraine le modèle sur toutes les données disponibles
pipe_text.fit(sms['sms'], y)

In [ ]:
# On va créer une fonction de filtre qui affiche un message si elle détecte un spam

def filtre_message(message):
    arr_mess = np.array([messsage])
    result = encode_y.inverse_transform(pipe_text.predict(arr_mess))[0]
    print(f'Ce message est : {result}')

In [ ]:
# Testons la fonction avec un novueau message

filtre_message("Urgent, You are a winner")